# Initialization

In [1]:
import time
from econia.account import Account
from econia.rest import EconiaClient, APT, USD, Buy, Sell

Initialize accounts

In [2]:
client = EconiaClient()
art = Account()
bud = Account()
econia = Account(dev_nb=True) # <-- for dev

Use aliases for shorter commands

In [3]:
art_ad = art.address()
bud_ad = bud.address()
u_ad = econia.address()
e_url = client.tx_vn_url_print
airdrop = client.airdrop_econia_coins
econia_balance = client.account_econia_coin_balances
transfer = client.transfer_econia_coins
deposit_econia = client.deposit_coins
c_balance = client.collateral_balances
w_u_coins = client.withdraw_coins

Fund all with TestCoin to cover tx fees

In [4]:
e_url(client.mint_testcoin(art_ad))

https://aptos-explorer.netlify.app/txn/159481


In [5]:
e_url(client.mint_testcoin(bud_ad))

https://aptos-explorer.netlify.app/txn/159485


In [6]:
e_url(client.mint_testcoin(u_ad))

https://aptos-explorer.netlify.app/txn/159490


# Airdrop and transfers

Publish Econia balance for Art

In [7]:
e_url(client.publish_econia_balances(art, u_ad))

https://aptos-explorer.netlify.app/txn/159494


Airdrop to Art: 100.3 `APT`, and 20,000.5 `USD`

In [8]:
e_url(airdrop(econia, art_ad, apt='100.3', usd='20000.5'))

https://aptos-explorer.netlify.app/txn/159498


Art's Regular balance has updated

In [9]:
econia_balance(art_ad, u_ad)

{'APT': Decimal('100.3'), 'USD': Decimal('20000.5')}

Publish Econia balance for Bud

In [10]:
e_url(client.publish_econia_balances(bud, u_ad))

https://aptos-explorer.netlify.app/txn/159502


Rather than airdropping, transfer 30.1 `APT` and 5,000.2 `USD` from Art to Bud

In [11]:
e_url(transfer(art, bud_ad, u_ad, apt='30.1', usd='5000.2'))

https://aptos-explorer.netlify.app/txn/159506


Bud's balance has gone up by 30.1 `APT` and 5000.2 `USD`

In [12]:
econia_balance(bud_ad, u_ad)

{'APT': Decimal('30.1'), 'USD': Decimal('5000.2')}

Art's balance has gone down by 30.1 `APT` and 5,000.2 `USD`, to 70.2 `APT` and 15,000.3 `USD`

In [13]:
econia_balance(art_ad, u_ad)

{'APT': Decimal('70.2'), 'USD': Decimal('15000.3')}

# Collateral account funding

Initialize Art's collateral account

In [14]:
e_url(client.init_account(art, u_ad))

https://aptos-explorer.netlify.app/txn/159510


His collateral balances are at 0

In [15]:
c_balance(art_ad, u_ad)

{'APT': {'holdings': Decimal('0'), 'available': Decimal('0')},
 'USD': {'holdings': Decimal('0'), 'available': Decimal('0')}}

Deposit 60.1 `APT` and 15,000.2 `USD` from Art's regular account to collateral

In [16]:
e_url(deposit_econia(art, u_ad, apt='60.1', usd='15000.2'))

https://aptos-explorer.netlify.app/txn/159514


Art's collateral balance has gone up by 60.1 `APT` and 15,000.2 `USD`, same with available amount

In [17]:
c_balance(art_ad, u_ad)

{'APT': {'holdings': Decimal('60.1'), 'available': Decimal('60.1')},
 'USD': {'holdings': Decimal('15000.2'), 'available': Decimal('15000.2')}}

Art's regular balance has gone down by 60.1 `APT` and 15,000.2 `USD` to 10.1 `APT` and 0.1 `USD`

In [18]:
econia_balance(art_ad, u_ad)

{'APT': Decimal('10.1'), 'USD': Decimal('0.1')}

Art withdraws 15 `APT` and 4,000 `USD` from collateral account

In [19]:
e_url(w_u_coins(art, u_ad, apt='15', usd='4000'))

https://aptos-explorer.netlify.app/txn/159519


Art's regular balance has increased by 15 `APT` and 4,000 `USD` to 25.1 `APT` and 4000.1 `USD`

In [20]:
econia_balance(art_ad, u_ad)

{'APT': Decimal('25.1'), 'USD': Decimal('4000.1')}

Art's collateral balances have decreased by 15 `APT` and 4,000 `USD` to 45.1 `APT` and 11,000 `USD`

In [21]:
c_balance(art_ad, u_ad)

{'APT': {'holdings': Decimal('45.1'), 'available': Decimal('45.1')},
 'USD': {'holdings': Decimal('11000.2'), 'available': Decimal('11000.2')}}

# Recording orders 

In [22]:
# Record mock orders to Art's account
r_m_o = client.record_mock_order

for id, side, price, unfilled in [
  # Various formats accepted
  # id   side   APT/USD   APT to fill
    (1,  Buy  ,   200    ,  '5.5' ),
    (2, 'Buy' ,  '190.5' ,  10    ),
    (3, 'Sell',  '230.0' ,  20    ),
    (4,  Buy  ,   180    ,  '7.25'),
    (5,  Sell ,   250    , '30.25')
]:
    e_url(r_m_o(econia, art_ad, id, side, price, unfilled))
    time.sleep(.5)

https://aptos-explorer.netlify.app/txn/159523
https://aptos-explorer.netlify.app/txn/159530
https://aptos-explorer.netlify.app/txn/159536
https://aptos-explorer.netlify.app/txn/159547
https://aptos-explorer.netlify.app/txn/159557


Orders have been recorded in Art's open orders

In [23]:
client.open_orders(art_ad, u_ad)

,side,price,unfilled
id,,,
1,Buy,200.0,5.50
2,Buy,190.5,10.00
3,Sell,230.0,20.00
4,Buy,180.0,7.25
5,Sell,250.0,30.25


# Matching orders 

In [24]:
t_m_o = client.trigger_match_order

Partially fill Art's order w/ id 1: Fill 5 of his limit buy's 5.5 `APT` at a price of 200 `USD` per `APT` (1,000 `USD` total)

In [25]:
e_url(t_m_o(econia, art_ad, id=1, apt=5, usd=0))

https://aptos-explorer.netlify.app/txn/159566


Order id 1 unfilled amount has decreased by 5 to 0.5

In [26]:
client.open_orders(art_ad, u_ad)

,side,price,unfilled
id,,,
1,Buy,200.0,0.50
2,Buy,190.5,10.00
3,Sell,230.0,20.00
4,Buy,180.0,7.25
5,Sell,250.0,30.25


Art's collateral balances have updated: 5 more `APT`, 1,000 less `USD`, putting him at 50.1 `APT` and 10,000.2 `USD`

In [27]:
c_balance(art_ad, u_ad)

{'APT': {'holdings': Decimal('50.1'), 'available': Decimal('50.1')},
 'USD': {'holdings': Decimal('10000.2'), 'available': Decimal('10000.2')}}

Completely fill Art's order w/ id 3: Fill all 20 of his limit sell's 20 `APT` at a price of 230 `USD` per `APT` (total of 4,600 `USD`)

In [28]:
e_url(t_m_o(econia, art_ad, id=3, apt=0, usd=4600))

https://aptos-explorer.netlify.app/txn/159575


Order id 3 is now gone from Art's orders

In [29]:
client.open_orders(art_ad, u_ad)

,side,price,unfilled
id,,,
1,Buy,200.0,0.50
2,Buy,190.5,10.00
4,Buy,180.0,7.25
5,Sell,250.0,30.25


Art's collateral balances have updated accordingly: 20 less `APT`, 4,600 more `USD`, putting him at 30.1 `APT` and 14,600.2 `USD`

In [30]:
c_balance(art_ad, u_ad)

{'APT': {'holdings': Decimal('30.1'), 'available': Decimal('30.1')},
 'USD': {'holdings': Decimal('14600.2'), 'available': Decimal('14600.2')}}